# (7A) Social networks

For this notebook, you'll need to install networkx. In terminal, type:

    pip install networkx
    pip install pyvis

In [ ]:
import json
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

## Introduction to networks

### History

Network theory has a famous beginning in a 1736 solution to a mathematical problem called the [Seven Bridges of Königsberg](https://en.wikipedia.org/wiki/Seven_Bridges_of_K%C3%B6nigsberg). The problem: is it possible to cross each of the seven bridges in the city of Königsberg only once?

<center><img src="Konigsberg_bridges.png" /></center>

A mathematician named [Leonhard Euler](https://en.wikipedia.org/wiki/Leonhard_Euler) proved in 1736 that the problem has no solution (it is not possible to cross each bridge only once). His solution laid the foundations of graph theory, because he discovered that the actual map and geographic space is irrelevant, and all we need to think about are nodes and edges.

<center><img src="konigsberg_to_network.png" /></center>

The seven bridges become seven "edges" between "nodes", and we can then demonstrate mathematically that there is no way to move through the network without crossing the same edge twice.

### Sample graph

In [ ]:
# Get a sample graph
g_karate = nx.karate_club_graph()

From the [source](http://vlado.fmf.uni-lj.si/pub/networks/data/Ucinet/UciData.htm#zachary): 

<blockquote>
BACKGROUND

These are data collected from the members of a university karate club by Wayne Zachary. The ZACHE matrix represents the presence or absence of ties among the members of the club; the ZACHC matrix indicates the relative strength of the associations (number of situations in and outside the club in which interactions occurred).

Zachary (1977) used these data and an information flow model of network conflict resolution to explain the split-up of this group following disputes among the members.

REFERENCE

Zachary W. (1977). An information flow model for conflict and fission in small groups. Journal of Anthropological Research, 33, 452-473.
</blockquote>

In [ ]:
# Draw graph
def draw_graph(g):
    fig, ax = plt.subplots(1, 1, figsize=(12, 8));
    nx.draw_networkx(g, ax=ax)

In [ ]:
draw_graph(g_karate)

In [ ]:
def draw_graph2(g):
    from pyvis import network as net
    G = net.Network(notebook=True)
    G.from_nx(g)
    G.show_buttons()
    return G.show('graph.html')

In [ ]:
#draw_graph2(g_karate)

In [ ]:
# Get the nodes
g_karate.nodes()

In [ ]:
# Get the edges
g_karate.edges()

### Network measures

#### Network degree

How many other nodes are connected to any given node? This number is called its "degree."

In [ ]:
# Get a dictionary from the node --> its "degree"
degree_dict = dict(g_karate.degree)
print(degree_dict)

In [ ]:
# Quick helper function to show a dictionary sorted by its value
def sort_dict(d,ascending=False):
    # convert the dictionary to a pandas "series" (effectively a dataframe column divorced from a dataframe)
    series = pd.Series(d)
    # return this series, sorted
    return series.sort_values(ascending=ascending)

In [ ]:
# Which nodes have the most "ties"?
sort_dict(degree_dict)

#### Network centrality

In [ ]:
# This is the same data except normalized by the total number of edges
degree_centrality = nx.degree_centrality(g_karate)

# Show
sort_dict(degree_centrality)

#### Betweeenness centrality

In [ ]:
betweenness_centrality = nx.betweenness_centrality(g_karate)
sort_dict(betweenness_centrality)

### Filtering graphs

In [ ]:
def filter_graph_by_degree(g,min_degree = 2):
    # make an empty list which will store all the nodes we want to remove
    nodes_to_remove = []
    
    # get the dictionary of node->degree
    degree_dict=dict(g.degree)
    
    # loop over the nodes
    for node in g.nodes():
        degree = degree_dict.get(node,0)
        if degree < min_degree:
            nodes_to_remove.append(node)
            
    # remove from network
    g.remove_nodes_from(nodes_to_remove)
    
    return g

In [ ]:
g=filter_graph_by_degree(g,2)

In [ ]:
draw_graph(g)

## Making networks from metadata

In [ ]:
# Here's a function to bring a google spreadsheet into a Pandas dataframe
def google2df(url):
    from io import BytesIO
    import requests
    r = requests.get(url)
    data = r.content
    df = pd.read_csv(BytesIO(data))
    return df.fillna('')

In [ ]:
# Corpus metadata for Tropic of Orange
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSwmeht7zqyfnQ_Z4LuDTDf4Zr_Ny38jfy0sYmRG4L6oxgH4MynYnhcnFzXq0yMMgPQ8TkuYsKm0Q0Q/pub?gid=0&single=true&output=csv'

In [ ]:
df = google2df(url)
df

In [ ]:
# Get this column
df['who talks to whom']

In [ ]:
# new graph
G_tropic_meta=nx.Graph()

# for each entry in this column
for entry in df['who talks to whom']:
    
    # for each comma
    for edge_str in entry.split(','):
        
        # split between characters
        chars = edge_str.split('<>')
        
        # if there are two things in between a "<>"
        if len(chars)==2:
            # get two chars
            char1 = chars[0].strip()
            char2 = chars[1].strip()
            
            # add edge between characters
            if not G_tropic_meta.has_edge(char1,char2):
                G_tropic_meta.add_edge(char1,char2,weight=1)
            else:
                G_tropic_meta[char1][char2]['weight']+=1

In [ ]:
# Draw this graph
draw_graph(G_tropic_meta)

## Making networks from NER

Let's make a network of the characters in Harry Potter.

In [ ]:
def ner_spacy(string):
    """
    Using spacy, this function takes any string, identifies the named entities in it,
    and returns a list of dictionaries, with one dictionary per named entitiy,
    where each dictionary looks like this:
    
    {
        'type': 'PERSON',
        'entity': 'Ryan',
        '_sent_num': 1,
        '_sent': 'Ryan Heuser cannot wait until he graduates from Stanford University.'
    }
    """
    
    try:
        # import spacy
        import spacy
    except ImportError:
        print("spacy not installed. Please follow directions above.")
        return

    # clean string
    string = string.strip().replace('\n',' ').replace("’","'").replace("‘","'")
    
    # load its default English model
    nlp = spacy.load("en_core_web_sm")

    # create a spacy text object
    doc = nlp(string)
    
    # make an output list
    output_list = []

    # loop over sentences
    sent_num=0
    for sent in doc.sents:
        sent_num+=1
        added_sent_already = False

        # loop over sentence's entities
        sent_doc = nlp(str(sent))
        for ent in sent_doc.ents:
            
            # make a result dict
            result_dict = {}
            
            # set sentence number
            result_dict['_sent_num'] = sent_num
            
            # store text too
            if not added_sent_already:
                result_dict['_sent'] = sent.text
                added_sent_already = True
            else:
                result_dict['_sent'] = ''
            
            # get type
            result_dict['type'] = ent.label_
            
            # get entity
            result_dict['entity'] = ent.text
            
            # get start char
            result_dict['start_char'] = ent.start_char
            
            # get end char
            result_dict['end_char'] = ent.end_char
            
            # add result_dict to output_list
            output_list.append(result_dict)
            
    # return output
    return output_list


In [ ]:
# set the path
text_path = '../corpora/harry_potter/texts/Sorcerers Stone.txt'
# text_path = '../corpora/tropic_of_orange/texts/ch01.txt'

limit_paragraphs = 100
#limit_paragraphs = None   # uncomment this to do whole text

# open the file
with open(text_path) as file:
    txt=file.read()

# make an empty network
G = nx.Graph()
    
# loop over the paragraphs!
paragraphs = txt.split('\n\n')

# randomize?
import random
random.shuffle(paragraphs)

# limit paragraphs?
paragraphs = paragraphs[:limit_paragraphs]

para_num=0
for para in paragraphs:
    para_num+=1
    if not para_num%10:
        print(para_num,len(paragraphs))
    
    # get the people for this paragraph
    people_in_this_para = []
    
    # get the NER results for this paragraph
    ner_results_ld = ner_spacy(para)
    
    # for each result
    for result_dict in ner_results_ld:
        # if it's a person:
        if result_dict['type']=='PERSON':
            # get the person's name
            person=result_dict['entity'].strip()
            
            # Let's do only first names
            if ' ' not in person:
                # add the person to the list
                people_in_this_para.append(person)
    
    ## get the unique pairs of persons in this paragraph
    # for each person1 in the paragraph
    for person1 in people_in_this_para:
        # for each person2 in the paragraph
        for person2 in people_in_this_para:
            # skip if we've repeated this already
            if person1>=person2: continue
            
            
            if not G.has_edge(person1,person2):
                G.add_edge(person1,person2,weight=1)
            else:
                G[person1][person2]['weight']+=1

In [ ]:
G_hp=filter_graph_by_degree(G,2)

In [ ]:
draw_graph(G_hp)

## Saving networks

### Graphml (good for Gephi)

Try downloading [Gephi](https://gephi.org/) and loading your graphml file for visualization.

In [ ]:
# Save as Graphml (good for Gephi)
nx.write_graphml(G, 'data.network.graphml')

### Edgelist (good for Palladio)

Try visualizing this in [Palladio](http://hdlab.stanford.edu/palladio-app).

In [ ]:
# Save as edgelist

def make_edge_table(g):
    results=[]
    for node1,node2,edge_data in g.edges(data=True):
        
        edge_data['source'] = node1
        edge_data['target'] = node2
        
        results.append(edge_data)
    
    return pd.DataFrame(results)

In [ ]:
edge_df = make_edge_table(G)
edge_df

In [ ]:
edge_df.to_csv('data.network.edges.csv',index=False)

## For network research team

* Generate a network from our metadata:
    * Divide up chapters among some people and annotate who speaks to whom [in our metadata spreadsheet](https://docs.google.com/spreadsheets/d/1cRmrwQmq2HuA-cb_mQYGGau40AO9fAROqRUK4EKKoJ4/edit?usp=sharing).
    * Generate the network from the metadata
    * Who is the most central character? Who is the most betweenness-central character?
    * Save the network
    * Visualize it in Palladio or Gephi


* Generate a network from NER:
    * Look at the code above, but adapt it to work for the whole novel
    * Visualize it in Gephi or Palladio    


* Generate networks by narrator:
    * Using either metadata or NER, which narrators' chapters create the most dense social networks? Which the most sparse? 
    
    
* (Advanced) Generate a network for other words
    * Maybe try this for place names also? Or for people AND placenames together?
    * Fruit? Plants? Commodities?


* Your own initiatives using networks and *Tropic of Orange*